## Import library

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import pyvi

## Setup data

### Data path

In [10]:
ViCTSD_train = "..\\Dataset\\ViCTSD\\ViCTSD_train.csv"
ViCTSD_test = "..\\Dataset\\ViCTSD\\ViCTSD_test.csv"
vihsd_train = "..\\Dataset\\vihsd\\train.csv"
vihsd_test = "..\\Dataset\\vihsd\\test.csv"

### Visualize data

* Visualize record
* Unique labels each path

In [11]:
ViCTSD_train_data = pd.read_csv(ViCTSD_train)[['Comment', 'Toxicity']]
ViCTSD_test_data = pd.read_csv(ViCTSD_test)[['Comment', 'Toxicity']]
vihsd_train_data = pd.read_csv(vihsd_train)
vihsd_test_data = pd.read_csv(vihsd_test)

### Get text and label each DataFrame

In [12]:
vihsd_train_data = vihsd_train_data.rename(columns={'free_text': 'Comment', 'label_id': 'Toxicity'})
vihsd_test_data = vihsd_test_data.rename(columns={'free_text': 'Comment', 'label_id': 'Toxicity'})

In [13]:
vihsd_train_data['Toxicity'] = vihsd_train_data['Toxicity'].replace(2, 1)
vihsd_test_data['Toxicity'] = vihsd_test_data['Toxicity'].replace(2, 1)

### Merge DataFrame

In [14]:
df_train_joined = pd.concat([ViCTSD_train_data, vihsd_train_data], ignore_index=True)
df_test_joined = pd.concat([ViCTSD_test_data, vihsd_test_data], ignore_index=True)

In [15]:
# Setup type for data in Comment column
df_train_joined['Comment'] = df_train_joined['Comment'].astype(str)
df_test_joined['Comment'] = df_test_joined['Comment'].astype(str)

### Check null and remove null data

In [16]:
# Lấy các cột có giá trị null
def check_null_record(dataframe: pd.DataFrame, column_name: str):
    columns_with_nulls = dataframe[column_name].isnull()
    null_comment_indices = columns_with_nulls[columns_with_nulls == True].index
    # print null dataframe
    print(dataframe.iloc[null_comment_indices])
    return null_comment_indices

In [17]:
check_null_record(df_train_joined, 'Comment')

Empty DataFrame
Columns: [Comment, Toxicity]
Index: []


Index([], dtype='int64')

In [18]:
df_train_joined.drop(check_null_record(df_train_joined, 'Comment'), inplace=True)

Empty DataFrame
Columns: [Comment, Toxicity]
Index: []


### Preprocess data function
* Tokenize: tuyệt vời -> tuyệt_vời, ngu ngốc -> ngu_ngốc
* Loại stopwords: Thật tuyệt_vời -> tuyệt_vời
* Loại link, hashtag và tag trong comment

In [ ]:
from pyvi import ViTokenizer, ViPosTagger
# word seqmentation
# ML không bắt buộc seqmentation
def tokenize(text):
    """
    Thật tuyệt vời -> Thật tuyệt_vời
    """
    return ViTokenizer.tokenize(text)

In [ ]:
import os
with open('Stopwords\\vietnamese-stopwords-dash.txt', 'r', encoding='utf-8') as f:
    # Đọc từng dòng trong file
    stopwords = [line.strip() for line in f]

In [ ]:
def lower_and_remove_stopwords(text):
  try:
    # Chuyển đổi văn bản sang chữ thường
    text = text.lower()

    # Tách văn bản thành danh sách các từ
    words = text.split()

    # Loại bỏ stopword
    filtered_words = [word for word in words if word not in stopwords]

    # Ghép danh sách các từ đã lọc lại thành văn bản
    filtered_text = ' '.join(filtered_words)
  except Exception as e:
    print(text)
  return filtered_text


In [ ]:
import re

def remove_links_hashtag_tag(text):
    """
    Removes URLs from a text string.

    Args:
        text: The text string to process.

    Returns:
        The text string with URLs removed.
    """
    link_remover = r"(https?://[^\s]+)"
    hashtag_remover = r"# [^\s]+"
    tag_remover = r"@ [^\s]+"

    text = re.sub(link_remover, "", text)
    text = re.sub(hashtag_remover, "", text)
    text = re.sub(tag_remover, "", text)
  
    return text

#### Test một dữ liệu khi qua quá trình chuẩn bị

In [ ]:
test_text = "#giải_cứu_bi_béo @Lê Tuấn tgia không bili, link: https://www.google.com"

In [ ]:
test_text = ViTokenizer.tokenize(test_text)
test_text

'# giải_cứu_bi_béo @ Lê_Tuấn tgia không bili , link : https://www.google.com'

In [ ]:
test_text = lower_and_remove_stopwords(test_text)
print(test_text)
test_text = remove_links_hashtag_tag(test_text)
test_text

# giải_cứu_bi_béo @ lê_tuấn tgia bili , link : https://www.google.com


'  tgia bili , link : '

In [ ]:
text = "Đây là một ví dụ về #hashtag, http://www.example.com và @username."
result = remove_links_hashtag_tag(text)
result

'Đây là một ví dụ về #hashtag,  và @username.'

## Create train, test

X_train, X_test, y_train, y_test

In [ ]:
def preprocess_data(path1, path2):
    """
    1. Get data from 2 paths
    2. Concat 2 dataframes
    3. Tokenize -> lower and remove stopwords -> remove links, hashtags, tags
    4. Get X, y from dataframe (input, output)
    """
    train_data1 = pd.read_csv(path1)[['Comment', 'Toxicity']]
    train_data2 = pd.read_csv(path2)
    train_data2 = train_data2.rename(columns={'free_text': 'Comment', 'label_id': 'Toxicity'})
    train_data2['Toxicity'] = train_data2['Toxicity'].replace(2, 1)
    df_joined = pd.concat([train_data1, train_data2], ignore_index=True)
    df_joined['Comment'] = df_joined['Comment'].astype(str)
    df_joined['Comment'] = df_joined['Comment'].apply(tokenize)
    df_joined['Comment'] = df_joined['Comment'].apply(lower_and_remove_stopwords)
    df_joined['Comment'] = df_joined['Comment'].apply(remove_links_hashtag_tag)
    X = df_joined['Comment']
    y = df_joined['Toxicity']
    return X, y, df_joined


In [ ]:
X, y, data = preprocess_data(ViCTSD_train, vihsd_train)
X_test, y_test, _ = preprocess_data(ViCTSD_test, vihsd_test)

In [ ]:
# Lưu dữ liệu sau khi đã xử lí
data.to_csv('E:\\AI Project\\VN-Hate-Speech-Detection-Personal\\df_joined.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

### a

In [50]:
vocab_size=50000
embedding_dim=64
max_length=140

In [111]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [112]:
# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

In [113]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
padded_X_train_sequences = pad_sequences(X_train_sequences, maxlen=max_length, padding='post',truncating='post')

In [117]:
# Thực hiện thay đổi test để đưa vào tính toán val_acc
test_sequences = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, padding='post',truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post',truncating='post')

In [120]:
# vocab_size=50000
# embedding_dim=64
# max_length=140

In [51]:
from torch import nn
class model(nn.Module):
    def __init__(self, vocab_size:int, embedding_dim:int):
        super().__init__()
        self.se = nn.Sequential(
            nn.Embedding(num_embeddings=vocab_size, 
                        embedding_dim=embedding_dim),
            nn.Flatten(),
            nn.Linear(in_features=embedding_dim, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.se(x)

In [1]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

NameError: name 'nn' is not defined

In [52]:
model = model(vocab_size=vocab_size, embedding_dim=embedding_dim)
model

model(
  (se): Sequential(
    (0): Embedding(50000, 64)
    (1): Flatten(start_dim=1, end_dim=-1)
    (2): Linear(in_features=64, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [53]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

'cpu'

In [ ]:
# Write a training and evaluationg loop for model_1
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Train for longer
epochs = 1000

# Put data on the target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test=  X_test.to(device), y_test.to(device)

for epoch in range (epochs):
  ### Training
  model.train()

  # 1. Forward
  y_logits = model(X_train).squeeze()
  y_pred = torch.round(torch.sigmoid(y_logits)) # logits -> pred probabilities -> predict

  # 2. Calculate the loss/acc
  loss = loss_fn(y_logits, y_train)
  acc = accuracy_fn(y_train,
                    y_pred)

  # 3. Optimizer zero grad
  optimizer.zero_grad()

  # 4. Loss backwards
  loss.backward()

  # 5. Optimizer step
  optimizer.step()

  ### Testing
  model.eval()
  with torch.inference_mode():
    # 1. Forward pass
    test_logits = model(X_test).squeeze()
    test_pred = torch.round(torch.sigmoid(test_logits))
    # 2. Calculate loss
    test_loss = loss_fn(test_logits,
                        y_test)
    test_acc = accuracy_fn(y_test,
                           test_pred)
  # Print out what happening
  if epoch%100==0:
    print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")


In [122]:
model.fit(padded_X_train_sequences, y_train, epochs=3, validation_data=(padded_test_sequences, y_val))

Epoch 1/3
874/874 [==============================] - 70s 77ms/step - loss: 0.3505 - acc: 0.8647 - val_loss: 0.3070 - val_acc: 0.8850
Epoch 2/3
874/874 [==============================] - 49s 56ms/step - loss: 0.2250 - acc: 0.9097 - val_loss: 0.3165 - val_acc: 0.8853
Epoch 3/3
874/874 [==============================] - 46s 53ms/step - loss: 0.1409 - acc: 0.9468 - val_loss: 0.3800 - val_acc: 0.8821


## Lưu model

In [123]:
# Saving our PyTorch model
from pathlib import Path

# 1. Create model directory
MODEL_PATH = "E:\\AI Project\\"

# 3. Save the model state dict
model.save('my_model.h5')

c:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Load model

In [124]:
from tensorflow.keras.models import load_model
loaded_model = load_model('my_model.h5')

## Dự đoán

### Nhập câu muốn dự đoán

In [125]:
test_sentences = [  "Đồ ngu, sao mày ngu thế?",
                    "Con này xấu xí, béo ú, chẳng ra gì cả.",
                    "Đồ đàn bà con gái chỉ biết ăn bám đàn ông.",
                    "Sản phẩm này tệ hại, ai mua cũng hối hận.",
                    "Mày hát dở tệ, đừng bao giờ hát nữa nhé.",
                    "Mày dám nói tao ngu à? Thử lặp lại lần nữa xem!",
                    "Đời mày xuống lỗ rồi con ạ",
                    "Ngu thì nín :))",
                    "khôn như vợ chồng nhà này k biết bao giờ mới giỗ đầu bạn nhỉ :))",
                    "toàn cái bọn lúc vay muốn dc vc.khi xong vc rồi thì vẩy đuôi b oi.giải thích bọn nó cố tính đéo hiểu đâu.còn vay gì trả đấy đơn giản chỉ thế.",
                    "sản phẩm này bố ỉa vào mà mua"
]

In [126]:
def predict(test_sentences):
    test_seq = tokenize(test_sentences)
    test_seq = lower_and_remove_stopwords(test_seq)
    test_seq = remove_links_hashtag_tag(test_seq)
    test_seq = tokenizer.texts_to_sequences([test_seq])
    # print(test_seq)
    test_seq = pad_sequences(test_seq, maxlen=max_length, padding='post',truncating='post')
    # print(test_seq.shape, test_seq)
    print(loaded_model.predict(test_seq))

In [127]:
for sentence in test_sentences:
    predict(sentence)

1/1 [==============================] - 0s 102ms/step
[[0.9667361]]
1/1 [==============================] - 0s 47ms/step
[[0.6757173]]
1/1 [==============================] - 0s 50ms/step


[[0.72355354]]
1/1 [==============================] - 0s 54ms/step
[[0.35801852]]
1/1 [==============================] - 0s 35ms/step
[[0.6409282]]
1/1 [==============================] - 0s 32ms/step
[[0.8147318]]
1/1 [==============================] - 0s 32ms/step
[[0.1019125]]
1/1 [==============================] - 0s 32ms/step
[[0.7510088]]
1/1 [==============================] - 0s 35ms/step
[[0.17122033]]
1/1 [==============================] - 0s 32ms/step
[[0.76874113]]
1/1 [==============================] - 0s 31ms/step
[[0.2812712]]


In [128]:
# "Đồ ngu, sao mày ngu thế?", 1
# "Con này xấu xí, béo ú, chẳng ra gì cả.", 1
# "Đồ đàn bà con gái chỉ biết ăn bám đàn ông.",
# "Sản phẩm này tệ hại, ai mua cũng hối hận.",
# "Mày hát dở tệ, đừng bao giờ hát nữa nhé.", 0 
# "Mày dám nói tao ngu à? Thử lặp lại lần nữa xem!",
# "Đời mày xuống lỗ rồi con ạ", 0
# "Ngu thì nín :))",
# "khôn như vợ chồng nhà này k biết bao giờ mới giỗ đầu bạn nhỉ :))", 0 
# "toàn cái bọn lúc vay muốn dc vc.khi xong vc rồi thì vẩy đuôi b oi.g
# "sản phẩm này bố ỉa vào mà mua"

In [129]:
# import csv
# filename = "E:\\AI Project\\word2vec_vi_syllables_100dims.txt"
# chunksize = 10 ** 6
# for chunk in pd.read_csv(filename, chunksize=chunksize, on_bad_lines='skip', quoting=csv.QUOTE_NONE, encoding='utf-8'):
#     # chunk is a DataFrame. To "process" the rows in the chunk:
#     for index, row in chunk.iterrows():
#         print(row)

In [130]:
predict = loaded_model.predict(X_test_padded)
predict = torch.tensor(predict)
predict = predict.round().type(torch.int64)
predict[:5]

240/240 [==============================] - 1s 5ms/step


tensor([[1],
        [0],
        [0],
        [0],
        [0]])

In [131]:
predict.squeeze()

tensor([1, 0, 0,  ..., 0, 1, 1])

In [132]:
predict.shape

torch.Size([7680, 1])

In [133]:
y_test = torch.tensor(y_test)
y_test

tensor([1, 1, 0,  ..., 0, 0, 0])

In [134]:
y_test.shape

torch.Size([7680])

In [135]:
ls = torch.eq(predict.squeeze(), y_test.squeeze())

In [136]:
cnt = 0
for i in ls:
    if i:
        cnt += 1
cnt

6770

In [137]:
6148/7680

0.8005208333333333

In [138]:
predict[0]==0

tensor([False])

In [139]:
print(torch.eq(predict.squeeze(), y_test.squeeze()))

tensor([ True, False,  True,  ...,  True, False, False])


In [140]:
y_train[-10:]

6265     0
22118    1
11284    0
11964    0
21575    0
29802    0
5390     0
860      0
15795    1
23654    0
Name: Toxicity, dtype: int64